In [8]:
import warnings
warnings.filterwarnings("ignore")

import requests
import pandas as pd
import bs4 as bs
import time

### **Hàm giúp chuyển đổi file data xml sang mảng 2 chiều giúp dễ dàng chuyển sang pd.Dataframe hơn**

In [4]:
def xml2df(xml_data):
    """ This function grabs the root of the XML document and iterates over
        the 'r' (row) and 'c' (column) tags of the data-table
        Rows with a 'v' attribute contain a numerical value
        Rows with a 'l attribute contain a text label and may contain an
        additional 'r' (rowspan) tag which identifies how many rows the value
        should be added. If present, that label will be added to the following
        rows of the data table.
    
        Function returns a two-dimensional array or data frame that may be 
        used by the pandas library."""
    
    root = bs.BeautifulSoup(xml_data,"lxml")
    all_records = []
    row_number = 0
    rows = root.find_all("r")
    
    for row in rows:
        if row_number >= len(all_records):
            all_records.append([])
              
        for cell in row.find_all("c"):
            if 'v' in cell.attrs:
                try:
                    all_records[row_number].append(float(cell.attrs["v"].replace(',','')))
                except ValueError:
                    all_records[row_number].append(cell.attrs["v"])
            else:
                if 'r' not in cell.attrs:
                    all_records[row_number].append(cell.attrs["l"])
                else:
                
                    for row_index in range(int(cell.attrs["r"])):
                        if (row_number + row_index) >= len(all_records):
                            all_records.append([])
                            all_records[row_number + row_index].append(cell.attrs["l"])
                        else:
                            all_records[row_number + row_index].append(cell.attrs["l"])
                                           
        row_number += 1
    return all_records

## **Thu thập dữ liệu từ năm 2003 đến năm 2006**

In [9]:
# Create an empty DataFrame with the specified columns plus a 'year' column
columns = ["Cause of death", "Age of mother", "Mother's education", "Birth weight", "Age of infant at death", "Deaths", "Births", "Death rate per 1000"]
dataset_2003_2006 = pd.DataFrame(columns=columns)
dataset_2003_2006['Year'] = pd.Series(dtype='int')

url = "https://wonder.cdc.gov/controller/datarequest/D31"

with open('request_2003_2006.xml', 'r') as file:
    lines = file.readlines()

for year in range(2003, 2007):
    lines[220] = f'\t\t<value>{str(year)}</value>\n'
    xml_request = ''.join(lines)

    response = requests.post(url, data={"request_xml": xml_request, "accept_datause_restrictions": "true"})

    if response.status_code == 200:
        data = response.text 
        data_frame = xml2df(data)

        new_data = pd.DataFrame(data=data_frame, columns=columns)
        new_data['Year'] = year  # Add the year column
        dataset_2003_2006 = pd.concat([dataset_2003_2006, new_data], ignore_index=True)
    else:
        print(f"Something went wrong for year {year}")

    # Wait for 90 seconds before making the next request
    time.sleep(90)

dataset_2003_2006.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4086 entries, 0 to 4085
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Cause of death          4086 non-null   object
 1   Age of mother           4086 non-null   object
 2   Mother's education      4086 non-null   object
 3   Birth weight            4086 non-null   object
 4   Age of infant at death  4086 non-null   object
 5   Deaths                  4086 non-null   object
 6   Births                  4086 non-null   object
 7   Death rate per 1000     4086 non-null   object
 8   Year                    4086 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 287.4+ KB


## **Thu thập dữ liệu từ năm 2007 đến năm 2022**

In [10]:
# Create an empty DataFrame with the specified columns plus a 'year' column
columns = ["Cause of death", "Age of mother", "Mother's education", "Birth weight", "Age of infant at death", "Deaths", "Births", "Death rate per 1000"]
dataset_2007_2022 = pd.DataFrame(columns=columns)
dataset_2007_2022['Year'] = pd.Series(dtype='int')

url = "https://wonder.cdc.gov/controller/datarequest/D69"

with open('request_2007_2022.xml', 'r') as file:
    lines = file.readlines()

for year in range(2007, 2023):
    lines[242] = f'\t\t<value>{str(year)}</value>\n'
    xml_request = ''.join(lines)

    response = requests.post(url, data={"request_xml": xml_request, "accept_datause_restrictions": "true"})

    if response.status_code == 200:
        data = response.text 
        data_frame = xml2df(data)

        new_data = pd.DataFrame(data=data_frame, columns=columns)
        new_data['Year'] = year  # Add the year column
        dataset_2007_2022 = pd.concat([dataset_2007_2022, new_data], ignore_index=True)
    else:
        print(f"Something went wrong for year {year}")

    # Wait for 100 seconds before making the next request
    time.sleep(100)

dataset_2007_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13187 entries, 0 to 13186
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Cause of death          13187 non-null  object
 1   Age of mother           13187 non-null  object
 2   Mother's education      13187 non-null  object
 3   Birth weight            13187 non-null  object
 4   Age of infant at death  13187 non-null  object
 5   Deaths                  13187 non-null  object
 6   Births                  13187 non-null  object
 7   Death rate per 1000     13187 non-null  object
 8   Year                    13187 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 927.3+ KB


### **Gộp hai khoảng thời gian dữ liệu đã thu thập**

In [11]:
dataset_2003_2022 =  pd.concat([dataset_2003_2006, dataset_2007_2022], ignore_index=True)
dataset_2003_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17273 entries, 0 to 17272
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Cause of death          17273 non-null  object
 1   Age of mother           17273 non-null  object
 2   Mother's education      17273 non-null  object
 3   Birth weight            17273 non-null  object
 4   Age of infant at death  17273 non-null  object
 5   Deaths                  17273 non-null  object
 6   Births                  17273 non-null  object
 7   Death rate per 1000     17273 non-null  object
 8   Year                    17273 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 1.2+ MB


### **Chuyển thành file csv**

In [12]:
dataset_2003_2022.to_csv('./DATA/infant_mortality_data_2003_2022.csv', index=False)

## **Lấy thêm các thuộc tính mở rộng liên quan đến chủ đề bộ dữ liệu từ năm 2017 đến năm 2022**

In [13]:
# Create an empty DataFrame with the specified columns plus a 'year' column
columns = ["Cause of death", "NICU Admission", "Antibiotics for Mother", "Mother's Pre-pregnancy BMI", "Infant birth weight", "Deaths", "Births", "Death rate per 1000"]
dataset_expanded = pd.DataFrame(columns=columns)
dataset_expanded['Year'] = pd.Series(dtype='int')

url = "https://wonder.cdc.gov/controller/datarequest/D159"

with open('request_2017_2022_expanded.xml', 'r') as file:
    lines = file.readlines()

for year in range(2017, 2023):
    lines[773] = f'\t\t<value>{str(year)}</value>\n'
    xml_request = ''.join(lines)

    response = requests.post(url, data={"request_xml": xml_request, "accept_datause_restrictions": "true"})

    if response.status_code == 200:
        data = response.text 
        data_frame = xml2df(data)

        new_data = pd.DataFrame(data=data_frame, columns=columns)
        new_data['Year'] = year  # Add the year column
        dataset_expanded = pd.concat([dataset_expanded, new_data], ignore_index=True)
    else:
        print(f"Something went wrong for year {year}")

    # Wait for 2 minutes before making the next request
    time.sleep(90)

In [7]:
dataset_expanded.info()
dataset_expanded.to_csv('./DATA/infant_mortality_data_expanded_2017_2022.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1443 entries, 0 to 1442
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Cause of death              1443 non-null   object
 1   NICU Admission              1443 non-null   object
 2   Antibiotics for Mother      1443 non-null   object
 3   Mother's Pre-pregnancy BMI  1443 non-null   object
 4   Infant birth weight         1443 non-null   object
 5   Deaths                      1443 non-null   object
 6   Births                      1443 non-null   object
 7   Death rate per 1000         1443 non-null   object
 8   Year                        1443 non-null   int64 
dtypes: int64(1), object(8)
memory usage: 101.6+ KB
